In [18]:
import pandas as pd
import numpy as np
import time
import re

In [5]:
# gp = pd.read_csv('../gameprograms230920.csv')
# gp['usage'] = '게임'

In [359]:
today = time.strftime('%y%m%d')
# pd.DataFrame(gp).to_csv(f'../gameprograms{today}.csv', index=None)

In [399]:
gr_origin = pd.read_csv('../gamerequirements230922.csv')

In [400]:
gr_origin

,name,cpu,ram,gpu,dx,os,sto,spec_class
0,A Bird Story,> Intel Pentium III 800 MHz,2 GB RAM,1024x768 High Color +,Version 9.0c,"XP, Vista, 7, 8",200 MB available space,0
1,A Blind Legend,Dual Core CPU,1 GB RAM,generally everything made since 2004 should work.,Version 9.0,Windows XP SP2+,400 MB available space,0
2,A Boy and His Blob,"AMD Athlon 64 X2 Dual Core Processor 4600+, 2....",1 GB RAM,NVIDIA GeForce 9600 GT 512 MB / ATI Radeon HD ...,NaN,Windows 7-32Bit or Windows 8-32Bit,3 GB available space,0
3,A Boy and His Blob,AMD Athlon 7850 Dual Core Processor 2.8GHz / I...,2 GB RAM,NVIDIA GeForce 9600 GT 1GB / ATI Radeon HD 465...,NaN,Windows 7-64Bit or Windows 8-64Bit,3 GB available space,1
4,A City Sleeps,Intel Core2 Duo 3.00GHz,2 GB RAM,ATI Radeon 2400,9.0c,Windows 7 / 8,NaN,0
...,...,...,...,...,...,...,...,...
12090,99Vidas,Intel Core i5 @ 2.66GHz / AMD Phenom,4 GB RAM,NVIDIA GeForce GTX 460 (1GB) / ATI Radeon HD 5...,Version 11,Microsoft Windows 7 SP2 / Windows 8.1 / Window...,2 GB available space,1
12091,9th Company: Roots of Terror,Intel Pentium 4 2400 MHz and higher,1024 Mb (2048 Mb for Windows Vista),3D accelerator with 128Mb video memory (NVIDIA...,DirectX 9.0 or higher,Windows XP SP2/Vista,NaN,0
12092,Ész-kerék,NaN,NaN,NaN,NaN,Dos,NaN,0
12093,été,NaN,NaN,NaN,NaN,Windows 7,4 GB available space,0


In [369]:
gr_origin['sto'].str.contains('4,500')

0        False
1        False
2        False
3        False
4          NaN
         ...  
12090    False
12091      NaN
12092      NaN
12093    False
12094    False
Name: sto, Length: 12095, dtype: object

In [370]:
gr_origin.loc[gr_origin['sto'].str.contains('3 meg of Hard Drive Space \(for shareware\)', na=False), 'sto']

Series([], Name: sto, dtype: object)

In [363]:
sto_regex_map = {',500': '500', '(a|A)t least ': '', '(M|m)b': 'MB', 'from': '', 'meg': 'MB',
                 'Hard Drive Space: ': '', 'Approximately ': '', '7200 RPM or SSD drive, ': '',
                 'Solid State Drive \(SSD\) ': '', 'Minimum ': '', '(a|A)bout ': '', 'Media: ': '',
                 'Hard disk with ': '', 'DirectX® 9\.0c': '', 'Up to ': '', '7200rpm or faster': '',
                 'Complete installation:': '', 'Installation requires ': '', 'approx\. ': '',
                 'Hard Drive Space: ': '', 'Hard disk drive with ': '', 'To install \~ ': '',
                 'SSD strongly recommended': '', 'Hard disk space ': '', 'We do not know the final size of our game yet': '',
                 'High Performance SSD Recommended with ': '', 'Please reserve ': '', 'SSD with ': '',
                 }
after_map = {r'(GB|Gb|[a-z]).*': ''}

In [364]:
sto = gr_origin.sto.replace(regex=sto_regex_map).replace(regex=after_map)

In [365]:
sto_list = []
for i in range(len(sto)):
    if pd.isnull(sto.loc[i]):
        sto_list.append(None)
        continue
    sto.loc[i] = sto.loc[i].replace(',', '.')
    mbflag = 'MB' in sto.loc[i]
    sto.loc[i] = re.sub('MB.*', '', sto.loc[i])
    sto.loc[i] = re.sub('[^\d.]', '', sto.loc[i])
    if mbflag: sto.loc[i] = str(float(sto.loc[i]) / 1000)
    if sto.loc[i] == '': sto.loc[i] = None
    sto_list.append(sto.loc[i])
    percent = i / len(gr_origin.sto) * 100
    print(f'\rgr: {i+1}/{len(gr_origin.sto)} ({percent:.0f}%)', end='')

gr: 12095/12095 (100%)

In [381]:
# for a, b in zip(gr_origin.sto, sto_list):
#     print(f'{a}::{b}')

In [393]:
gr_origin.ram.unique()

array(['2 GB RAM', '1 GB RAM', '512 MB', ..., '512 MB or more of RAM ',
       '2GB RAM (Windows XP) ', '1024 Mb (2048 Mb for Windows Vista)'],
      dtype=object)

In [394]:
ram_regex_map = {}
after_map = {r'(GB|Gb|[a-z]).*': ''}

In [395]:
ram = gr_origin.ram.replace(regex=ram_regex_map).replace(regex=after_map)

In [405]:
ram_list = []
for i in range(len(ram)):
    if pd.isnull(ram.loc[i]):
        ram_list.append(None)
        continue
    ram.loc[i] = ram.loc[i].replace(',', '.')
    mbflag = 'MB' in ram.loc[i]
    ram.loc[i] = re.sub('MB.*', '', ram.loc[i])
    ram.loc[i] = re.sub('[^\d.]', '', ram.loc[i])
    if mbflag: ram.loc[i] = str(float(ram.loc[i]) / 1024)
    if ram.loc[i] == '': ram.loc[i] = None
    ram_list.append(ram.loc[i])
    percent = i / len(gr_origin.ram) * 100
    print(f'\rgr: {i+1}/{len(gr_origin.ram)} ({percent:.0f}%)', end='')

gr: 12095/12095 (100%)

In [397]:
gr_origin.ram.loc[956] = '1'

In [402]:
gr_origin.ram.loc[956]

'512+MB of RAM'

In [391]:
pd.DataFrame(gr_origin).to_csv(f'../gamerequirements{today}.csv', index=None)

In [406]:
for a, b in zip(gr_origin.ram, ram_list):
    print(f'{a}::{b}')

2 GB RAM::2
1 GB RAM::1
1 GB RAM::1
2 GB RAM::2
2 GB RAM::2
512 MB::0.5
1 GB::1
2 GB RAM::2
1024 MB (XP) / 2048 MB (VISTA/7) ::1.0
2 GB RAM::2
4 GB RAM::4
2 GB RAM::2
4 GB RAM::4
4 GB RAM::4
8 GB RAM::8
1 GB RAM::1
4 GB RAM::4
4 GB RAM::4
4 GB RAM::4
8 GB RAM::8
4 GB RAM::4
8 GB RAM::8
6 GB RAM::6
8 GB RAM::8
64 MB RAM::0.0625
1GB RAM (Vista/7: 1,5GB RAM)::1
4 GB RAM::4
2 GB RAM::2
8 GB RAM::8
16 GB RAM::16
16 GB::16
16 GB::16
2 GB RAM::2
4 GB RAM::4
6 GB RAM::6
6 GB RAM::6
1 GB RAM::1
2 GB RAM::2
4 GB RAM::4
4 GB RAM::4
2 GB RAM::2
4 GB RAM::4
4 GB RAM::4
8 GB RAM::8
512 MB::0.5
512 MB::0.5
nan::None
nan::None
2 GB RAM::2
8 GB::8
16 GB::16
8 GB RAM::8
4 GB RAM::4
8 GB RAM::8
4 GB RAM::4
8 GB RAM::8
256 MB ::0.25
512 Mb::512
256 MB::0.25
768 MB::0.75
2 GB RAM::2
3 GB RAM::3
128 MB RAM::0.125
512 MB RAM::0.5
1GB::1
4 GB RAM::4
1,5 Go for windows XP / 2 Go for Vista, Seven or 8 GB RAM ::15
2 Go for 32 bit windows (XP/Vista/7/8) / 3 Go for 64 bit windows (XP/Vista/7/8) GB RAM::2
4 GB RAM: